In [36]:
import webvtt
from os import listdir
from os.path import isfile, join
import pandas as pd
from pydub import AudioSegment
import os
import shutil
import natsort 
import wave
import xml.etree.ElementTree as ET

In [38]:
rootdir= "/home/megha/Desktop/b"

# 1) # make new directory and convert the wav file
#---------------------------------------------------------------------------------------------------------------
if False: #False  True
    i= 0
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        if not os.path.exists(rootdir+"/"+Dir+"/temp"): 
            os.makedirs(rootdir+"/"+Dir+"/temp")
        i = i+1
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
            if(file.endswith(".WAV")):
                actual_filename = "folk_"+str(i)
                a = os.system('ffmpeg -i "{}" -acodec pcm_s16le -ac 1 -ar 16000 "{}/{}".wav'.format(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/temp", actual_filename))# (source,target(half path),name_of_file)
                print(a)
                if(a==256):
                        print("Did not get converted")
            if(file.endswith(".fln")):
                shutil.copyfile(rootdir+"/"+Dir+"/"+file, rootdir+"/"+Dir+"/temp/"+file)
            
#2) extract information from .fln file
#---------------------------------------------------------------------------------------------------------------
if False: #False  True
    for Dir in natsort.natsorted(os.listdir(rootdir)):        
        fileName=""
        #df  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df1  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df2  = pd.DataFrame(columns = ['A', 'B', 'C'])
        df3  = pd.DataFrame(columns = ['A', 'B', 'C'])
        for file in os.listdir(rootdir+"/"+Dir+"/temp"):            
            if(file.endswith(".wav")):
                fileName = file.split('.')[-2]+"_"
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
            #print(file)
            if(file.endswith(".fln")):                
                if(file.endswith(".fln") and file.split('_')[-1]=="01.fln" ):
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        point = 0
                        for child in children:   
                            if(child.tag == "time"):
                                if(point == 0):
                                    time_array.append(child.attrib['time'])
                                if(point == 1):
                                    time_array.append(child.attrib['time'])
                                    df_a = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))                            
                                    df1 = df1.append(df_a,ignore_index = True)
                                    word = ""
                                    time_array[0] = time_array[-1]  
                                    point = 0
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            if(child.tag == "boundary"):
                                point = 1
                                continue                                             
                #print(df1.size)
                if(file.endswith(".fln") and file.split('_')[-1]=="02.fln" ): 
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:   
                            if(child.tag == "time"):
                                if(point == 0):
                                    time_array.append(child.attrib['time'])
                                if(point == 1):
                                    time_array.append(child.attrib['time'])
                                    df_b = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))                            
                                    df2 = df2.append(df_b,ignore_index = True)
                                    word = ""
                                    time_array[0] = time_array[-1]  
                                    point = 0
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            if(child.tag == "boundary"):
                                point = 1
                                continue  
                if(file.endswith(".fln") and file.split('_')[-1]=="03.fln" ): 
                    tree = ET.parse(rootdir+"/"+Dir+"/temp/"+file)
                    root = tree.getroot()
                    for contribution in root.findall("./contribution"):
                        children = contribution.getchildren()
                        time_array = []  
                        word = ""
                        for child in children:   
                            if(child.tag == "time"):
                                if(point == 0):
                                    time_array.append(child.attrib['time'])
                                if(point == 1):
                                    time_array.append(child.attrib['time'])
                                    df_c = pd.DataFrame([[time_array[0],time_array[-1],word]], columns=list('ABC'))                            
                                    df3 = df3.append(df_c,ignore_index = True)
                                    word = ""
                                    time_array[0] = time_array[-1]  
                                    point = 0
                            if(child.tag == "uncertain"):
                                for c in child:
                                    word = word+" "+c.text
                            if(child.tag == "w"):
                                word = word+" "+child.text
                            if(child.tag == "boundary"):
                                point = 1
                                continue  
                #print(df3.size) 
        frames = []
        frames = [df1, df2, df3]
        df = pd.concat(frames)
        df.reset_index(drop=True, inplace=True)
        #check = df.iloc[::1, :].index
        time1 = []
        time2=  []
        starttime = []
        endtime = []
        string = []
        string_final = []
        for i in range(len(df)):
            x = df.get_values()[i]
            time1.append(float(x[0]))
            time2.append(float(x[1]))
            string.append(x[2])
            starttime.append(time1[-1])
            endtime.append(time2[-1])
            string_final.append(string[-1]) 
        string_final =  [x.upper() for x in string_final] 
        df_final = pd.DataFrame({'col1':starttime, 'col2':endtime, 'col3':string_final})
        df_final = df_final.drop(df_final[df_final.col3 ==""].index)         
        filename_list = []
        for i in range(0, len(df_final)):                    
            filename_list.extend([fileName+str(i)])    
        df_final['col4'] = filename_list
        print(df_final.shape)
        #----------------------------txt file creation-----------------------------------------------------------------   
        with open(rootdir+"/"+Dir+"/temp/"+"folk.trans.txt", "a") as file:
            for i in range(0, len(df_final)):
                file.write(df_final['col4'].iloc[i]+" "+df_final['col3'].iloc[i]+"\n")
        #----------------------------chuncking-----------------------------------------------------------------   
        for file1 in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp/")):
            if(file1.endswith(".wav")):
                myaudio = AudioSegment.from_file(rootdir+"/"+Dir+"/temp/"+file1, "wav")
                for i in range(0, len(df_final)):
                    START = (df_final['col1'].iloc[i] * 1000)
                    END = (df_final['col2'].iloc[i] * 1000)
                    chunk = myaudio[START:END]     
                    chunk_name = df_final['col4'].iloc[i]+".flac" 
                    if not os.path.exists(rootdir+"/"+Dir+"/temp/"+chunk_name):
                        print("chunking the audio", chunk_name)
                        chunk.export(rootdir+"/"+Dir+"/temp/"+chunk_name, format="wav")
            
#3) --------------------------------------cleaning folder---------------------------------------------
if False: # False True
    for Dir in natsort.natsorted(os.listdir(rootdir)):
        #print(Dir)
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir)):
            print(file)
            if(file.endswith(".xml") or file.endswith(".WAV") or file.endswith(".fln") or file.endswith(".html")):
                os.remove(rootdir+"/"+Dir+"/"+file)
        for file in natsort.natsorted(os.listdir(rootdir+"/"+Dir+"/temp")):
            print(file)
            if(file.endswith(".fln") or file.endswith(".wav")):
                os.remove(rootdir+"/"+Dir+"/temp/"+file)
            else:
                shutil.move(rootdir+"/"+Dir+"/temp/"+file, rootdir+"/"+Dir+"/"+file)
        os.rmdir(rootdir+"/"+Dir+"/temp/")


PF--_E_00005.xml
PF--_E_00005_SE_01_A_01_DF_01.WAV
PF--_E_00005_SE_01_T_01.fln
PF--_E_00005_SE_01_T_01.html
PF--_E_00005_e_doc.html
PF--_S_00005.xml
temp
PF--_E_00005_SE_01_T_01.fln
folk.trans.txt
folk_1.wav
folk_1_0.flac
folk_1_1.flac
folk_1_2.flac
folk_1_3.flac
folk_1_4.flac
folk_1_5.flac
folk_1_6.flac
folk_1_7.flac
folk_1_8.flac
folk_1_9.flac
folk_1_10.flac
folk_1_11.flac
folk_1_12.flac
folk_1_13.flac
folk_1_14.flac
folk_1_15.flac
folk_1_16.flac
folk_1_17.flac
folk_1_18.flac
folk_1_19.flac
folk_1_20.flac
folk_1_21.flac
folk_1_22.flac
folk_1_23.flac
folk_1_24.flac
folk_1_25.flac
folk_1_26.flac
folk_1_27.flac
folk_1_28.flac
folk_1_29.flac
folk_1_30.flac
folk_1_31.flac
folk_1_32.flac
folk_1_33.flac
folk_1_34.flac
folk_1_35.flac
folk_1_36.flac
folk_1_37.flac
folk_1_38.flac
folk_1_39.flac
folk_1_40.flac
folk_1_41.flac
folk_1_42.flac
folk_1_43.flac
folk_1_44.flac
folk_1_45.flac
folk_1_46.flac
folk_1_47.flac
folk_1_48.flac
folk_1_49.flac
folk_1_50.flac
folk_1_51.flac
folk_1_52.flac
folk_1_

PF--_E_00088_SE_01_T_01.fln
PF--_E_00088_SE_01_T_01.html
PF--_E_00088_e_doc.html
PF--_S_00088.xml
temp
PF--_E_00088_SE_01_T_01.fln
folk.trans.txt
folk_11.wav
folk_11_0.flac
folk_11_1.flac
folk_11_2.flac
folk_11_3.flac
folk_11_4.flac
folk_11_5.flac
folk_11_6.flac
folk_11_7.flac
folk_11_8.flac
folk_11_9.flac
folk_11_10.flac
folk_11_11.flac
folk_11_12.flac
folk_11_13.flac
folk_11_14.flac
folk_11_15.flac
folk_11_16.flac
folk_11_17.flac
folk_11_18.flac
folk_11_19.flac
folk_11_20.flac
folk_11_21.flac
folk_11_22.flac
folk_11_23.flac
folk_11_24.flac
folk_11_25.flac
folk_11_26.flac
folk_11_27.flac
folk_11_28.flac
folk_11_29.flac
folk_11_30.flac
folk_11_31.flac
folk_11_32.flac
folk_11_33.flac
folk_11_34.flac
folk_11_35.flac
folk_11_36.flac
folk_11_37.flac
folk_11_38.flac
folk_11_39.flac
folk_11_40.flac
folk_11_41.flac
folk_11_42.flac
folk_11_43.flac
folk_11_44.flac
folk_11_45.flac
folk_11_46.flac
folk_11_47.flac
folk_11_48.flac
folk_11_49.flac
folk_11_50.flac
folk_11_51.flac
folk_11_52.flac
folk

folk_23_0.flac
folk_23_1.flac
folk_23_2.flac
folk_23_3.flac
folk_23_4.flac
folk_23_5.flac
folk_23_6.flac
folk_23_7.flac
folk_23_8.flac
folk_23_9.flac
folk_23_10.flac
folk_23_11.flac
folk_23_12.flac
folk_23_13.flac
folk_23_14.flac
folk_23_15.flac
folk_23_16.flac
folk_23_17.flac
folk_23_18.flac
folk_23_19.flac
folk_23_20.flac
folk_23_21.flac
folk_23_22.flac
folk_23_23.flac
folk_23_24.flac
folk_23_25.flac
folk_23_26.flac
folk_23_27.flac
folk_23_28.flac
folk_23_29.flac
folk_23_30.flac
folk_23_31.flac
folk_23_32.flac
folk_23_33.flac
folk_23_34.flac
folk_23_35.flac
folk_23_36.flac
folk_23_37.flac
folk_23_38.flac
folk_23_39.flac
folk_23_40.flac
folk_23_41.flac
folk_23_42.flac
folk_23_43.flac
folk_23_44.flac
folk_23_45.flac
folk_23_46.flac
folk_23_47.flac
folk_23_48.flac
folk_23_49.flac
folk_23_50.flac
folk_23_51.flac
folk_23_52.flac
folk_23_53.flac
folk_23_54.flac
folk_23_55.flac
folk_23_56.flac
folk_23_57.flac
folk_23_58.flac
folk_23_59.flac
folk_23_60.flac
folk_23_61.flac
folk_23_62.flac
fo

In [34]:
#df = df[['A','B', 'C']]
df.reset_index(drop=True, inplace=True)



In [34]:
pd.set_option('display.max_colwidth', -1)
df_final

,col1,col2,col3,col4
0,0.17,6.430,HERR SCH SIE SIND IN VELTHEIM AN DER OHE GEBOREN UND HABEN DORT IHRE JUGEND VERBRACHT,folk_1_0
1,6.43,11.580,SIE HABEN SICHER NOCH ALLERLEI ERINNERUNGEN AN ALTE VOLKSBRÄUCHE,folk_1_1
2,11.58,13.810,KÖNNEN SIE UNS DARÜBER MAL ETWAS ERZÄHLEN,folk_1_2
3,13.81,23.950,JA ICH BIN IN MEINER HEIMAT VELTHEIM AN DER OHE GEBOREN UND HABE DA AUCH MEINE JUGEND ZUM TEIL VERLEBT,folk_1_3
4,23.95,27.770,WIE ICH DENN ÄLTER WURDE BIN ICH JA AUS DEM DORF RAUSGEGANGEN,folk_1_4
5,27.77,34.890,ABER SONST WIE ES SO FRÜHER AUF DEN DÖRFERN WAR ES WAR NOCH NICHT MODERN,folk_1_5
6,34.89,36.610,ICH KAM AUF 'NE VOLKSSCHULE,folk_1_6
7,36.61,50.420,DA WURDE DENN UNSER LEHRER ZUM TEIL NOCH MIT ONKEL ANGEREDET UND AUCH UNSERE KLASSE DIE WAR MANCHMAL ACHTZIG MANN STARK WOHL ACHTZIG KINDER STARK,folk_1_7
8,50.42,54.610,UND WIR KRIEGTEN DANN ERST EINE KLEINE SCHULE,folk_1_8
9,54.61,59.890,ALSO WIR HATTEN VORMITTAGS SCHULE UND NACHMITTAGS SCHULE,folk_1_9


In [11]:
df.to_csv("/home/megha/Desktop/c/01.csv", sep='\t', encoding='utf-8')